In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
%pip install -qU langchain-google-vertexai
%pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import libraries:
# I'm following the LangChain documentation here: https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
import getpass
import os

# It will ask for your OpenAI key here

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# Identical to what we already had:
# Read pdfs into the vector database

from langchain_community.document_loaders import PyPDFLoader

# def load_chunk_pdf(path= "../GenAI/"):
def load_chunk_pdf(path= "/content/gdrive/MyDrive/Colab Notebooks/Side Hustling 🤓"):
    pdf_folder_path = path
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):

            print (file)
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    splits = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

    return vectorstore

vectorstore = load_chunk_pdf()
retriever = vectorstore.as_retriever()

presente-indicativo.pdf
ACT.PRESENTE_V. IRREGULARES_CAMBIOS ORTOGRAFÍCOS.pdf
Presente de indicativo.pdf
ACT. VERBOS REFLEXIVOS.pdf
Verbos REFLEXIVOS y lista de más comunes.pdf
Los verbos reflexivos en español.pdf
Peregrinación a Santiago B1.pdf


In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate

# Here you can fine tune your Prompt. (note this is prompt adjustments from ChatGPT suggestions)

system_prompt = (
    "You are Ollie, a friendly, witty, and informed Spanish teaching chatbot! "\
    "Your goal is to help the user learn Spanish based on their personal goals and scenarios. "\
    "To start, introduce yourself as Ollie and explain that you’ll be teaching them Spanish in the contexts that matter most to them. "\
    "Since the user is a native English speaker, make sure to explain concepts in English before diving into Spanish. "\
    "Ask the user why they are learning Spanish—whether it’s 'for fun', 'for living', or 'for a trip'. "\
    "Next, ask them what regional Spanish they want to focus on: Spain, Mexico, Argentina, or another region. "\
    "After that, inquire about specific topics or scenarios they’d like to learn for, such as activities on their trip. "\
    "Personalize all future lessons based on their region and their interests, and stick to those scenarios throughout the lessons. "\
    "Before starting the lessons, tell the user 'Before jumping in, let’s figure out your current Spanish level!' and assess their proficiency through a few questions. "\
    "Once the level is determined, let them know what level they’re at and customize the teaching to that level. "\
    "Now say 'Let’s begin!' and explain that the lesson will have five fun and engaging questions, including fill-in-the-blank, matching, and open-ended responses. "\
    "Only ask questions relevant to the scenarios they chose. After each answer, explain why it’s right or wrong. "\
    "After the lesson, celebrate their progress and ask if they’d like to try a role-play session based on what they’ve learned. "\
    "Guide them through a fun, realistic role play, getting into character based on the scenario they chose. "\
    "Once the role play is done, recap their learning and give feedback. "\
    "Remember, your focus is teaching Spanish, and you cannot assist with other topics."\
    "\n\n"
"{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [11]:
## Adding chat history

# This allows to make back references to previous questions, so Cass' answers will have "memory"

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "\
    "which might reference context in the chat history, "\
    "formulate a standalone question which can be understood "\
    "without the chat history. Do NOT answer the question, "\
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [12]:
# Adding conversation sessions to save and continue conversations

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

**Testing with StreamLit**

In [15]:
import os

# Set the USER_AGENT environment variable
os.environ["USER_AGENT"] = "Ollie"  # Customize this string as needed

# Rest of your Streamlit code...



In [14]:
import streamlit as st

# Set page config for better mobile experience
st.set_page_config(page_title="Ollita - Your Spanish Tutor", page_icon="🧑‍🏫", layout="centered")

# UI for the chatbot
st.markdown("<h1 style='color: lightpurple; font-size: 30px;'>Ollita - Your Spanish Tutor</h1>", unsafe_allow_html=True)
st.markdown("<p style='color: black;'>Hi! I'm Ollie, and I'm here to teach you Spanish based on what matters most to you!</p>", unsafe_allow_html=True)

# Adding the chatbot interaction box
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Text box for user input
user_input = st.text_input("Ask Ollie anything about learning Spanish:")

# Triggering the chatbot
if user_input:
    # Append user message to chat history
    st.session_state.chat_history.append(("You", user_input))

    # Call your bot's backend function
    response = rag_chain({"input": user_input})

    # Add bot response to chat history
    st.session_state.chat_history.append(("Ollie", response["answer"]))

# Display chat history
for message in st.session_state.chat_history:
    user, msg = message
    if user == "Ollie":
        st.markdown(f"<p style='background-color: lightpurple; padding: 10px; border-radius: 10px;'>{user}: {msg}</p>", unsafe_allow_html=True)
    else:
        st.markdown(f"<p style='background-color: lightgray; padding: 10px; border-radius: 10px;'>{user}: {msg}</p>", unsafe_allow_html=True)

# Add a clear button for resetting the chat history
if st.button("Clear"):
    st.session_state.chat_history = []


2024-09-25 19:48:01.983 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:48:01.985 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:48:02.052 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-25 19:48:02.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:48:02.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:48:02.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:48:02.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

**Testing FE with Gradio**

In [17]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1


In [18]:
# User interface with Gradio
# Change debug --> False and comment "prints"

import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []

    def user (user_message, history):
        #print("user", user)
        #print("history", history)
        response= conversational_rag_chain.invoke(
            {"input": user_message},
            config={"configurable": {"session_id": "Test"}})
        #print(response['answer'])
        history.append((user_message, response["answer"]))
        #print("Updated chat history", history)
        return gr.update(value=""), history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue= False)
    clear.click(lambda: None, None, chatbot, queue= False)

if __name__ == "__main__":
    demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0935b451436f6b6ddc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0935b451436f6b6ddc.gradio.live


In [19]:
import gradio as gr

# Backend logic for chatbot (connected to your conversational_rag_chain backend)
def user(user_message, history):
    # Call your conversational chain (replace with your actual backend call)
    response = conversational_rag_chain.invoke(
        {"input": user_message},
        config={"configurable": {"session_id": "Test"}}
    )
    # Append user message and chatbot response to history
    history.append((user_message, response["answer"]))
    return gr.update(value=""), history

# Custom CSS for UI enhancements
css = """
    body {
        background-color: white;
        font-family: 'Ubuntu', sans-serif;
    }
    .gradio-container {
        background-color: white;
    }
    .gradio-btn-primary {
        background-color: lightgreen;
        color: black;
    }
    .gradio-chatbot .gr-chat-message-user {
        background-color: #d3c8e0; /* Light purple for user messages */
        color: black;
    }
    .gradio-chatbot .gr-chat-message-bot {
        background-color: #e1c6e6; /* Light purple for bot messages */
        color: black;
    }
    .logo {
        width: 60px; /* Set the desired width */
        height: auto; /* Maintain aspect ratio */
    }
    h1 {
        font-size: 24px;
        font-weight: bold;
        margin-left: 20px; /* Space between logo and heading */
        margin-bottom: 20px;
        font-family: 'Ubuntu', sans-serif;
        color: #b19cd9; /* Light purple for heading text */
        display: inline-block; /* Makes the heading inline with the logo */
        vertical-align: middle; /* Aligns the text vertically with the logo */
    }
"""

# Gradio UI layout
with gr.Blocks(css=css) as demo:
    # Add a row for logo and heading
    with gr.Row():
        # Add the logo without the label
        gr.Image("/content/Screenshot 2024-09-20 at 2.57.23 PM.png", label=None, elem_id="logo", type="filepath", elem_classes="logo")

        # Add the heading next to the logo
        gr.Markdown("<h1>Hi, I'm Ollie! I am here to assist you in learning a language customized to your needs!</h1>")

    # Chatbot interface without labels
    chatbot = gr.Chatbot(label=None)
    msg = gr.Textbox(placeholder="Type your message here...", label=None)

    # Clear and Send buttons
    clear = gr.Button("Clear", variant="primary")
    send_button = gr.Button("Send", variant="primary")

    chat_history = []

    # Function to submit the message (on pressing 'Send' button or 'Enter')
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    send_button.click(user, [msg, chatbot], [msg, chatbot], queue=False)

    # Clear the chat history when "Clear" button is clicked
    clear.click(lambda: None, None, chatbot, queue=False)

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d32f733aff9125bf74.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
